In [2]:
from sqlalchemy import create_engine
import pandas as pd
import os
from flask import Flask
from flask_sqlalchemy import SQLAlchemy

#engine = create_engine('sqlite:///recommender5.db', echo=False)
#create sqlite engine connected to flask
app = Flask(__name__)

engine = create_engine('sqlite:///recommender5.db', echo=False)

for f in os.listdir('data/movies/ml-latest-small'):
    if f[-4:] == '.csv':
        data = pd.read_csv(f'data/movies/ml-latest-small/{f}')
        data.to_sql(f[:-4], engine)
        print(f[0:-4])

links
ratings
movies
tags


In [3]:
movie_id_unique = 'SELECT * FROM movies'
all_movies = pd.read_sql(movie_id_unique, engine)

In [4]:
watched_movie_id_list = ['70286', '109487', '589']
#เอาuser input มาจาก flask

In [6]:
import numpy as np

#remove the input movies from all_movies
movies_not_watched = all_movies[~all_movies['movieId'].isin(watched_movie_id_list)]
movies_not_watched['fake_id'] = np.ones(len(movies_not_watched), dtype =int)

#get all_ratings from sqlite
query = 'SELECT "userId", ratings."movieId", movies.title, rating FROM ratings JOIN movies ON ratings."movieId" = movies."movieId";'
all_ratings = pd.read_sql(query, engine)

#remove the watched movies from all_ratings
not_all_ratings = all_ratings[~all_ratings['movieId'].isin(watched_movie_id_list)]

#remove movieId and ratings of the watched movies???
movieindex = not_all_ratings['movieId'].unique().tolist()
dl_movie2movie_encoded = {x: i for i, x in enumerate(movieindex)}
dl_movie_encoded2movie = {i: x for i, x in enumerate(movieindex)}

not_all_ratings["movie"] = not_all_ratings["movieId"].map(dl_movie2movie_encoded)
not_all_ratings["rating"] = not_all_ratings["rating"].values.astype(np.float32)

#map userId from ?? to not_all_ratings ?? what do i do with the users??

not_all_user_ids = not_all_ratings["userId"].unique().tolist()
dl_user2user_encoded = {x: i for i, x in enumerate(not_all_user_ids)}
dl_userencoded2user = {i: x for i, x in enumerate(not_all_user_ids)}

not_all_ratings["user"] = not_all_ratings["userId"].map(dl_user2user_encoded)

min_rating = min(not_all_ratings["rating"])
max_rating = max(not_all_ratings["rating"])
num_users = len(dl_user2user_encoded)
num_movies = len(dl_movie_encoded2movie)
print(
        "Number of users: {}, Number of Movies: {}, Min rating: {}, Max rating: {}".format(
            num_users, num_movies, min_rating, max_rating
        )
    )

#define training data
df = not_all_ratings.sample(frac=1, random_state=42)
x = not_all_ratings[["user", "movie"]].values

    # Normalize the targets between 0 and 1. Makes it easy to train.
y = not_all_ratings["rating"].apply(lambda x: (x - min_rating) / (max_rating - min_rating)).values
    # Assuming training on 90% of the data and validating on 10%.
train_indices = int(0.9 * df.shape[0])
x_train, x_val, y_train, y_val = (
        x[:train_indices],
        x[train_indices:],
        y[:train_indices],
        y[train_indices:],
    )

#inference
user_movie_array = movies_not_watched[['fake_id','index']]
max_movie_index = user_movie_array['index'].max()
that = user_movie_array.to_numpy()

EMBEDDING_SIZE = 50

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class RecommenderNet(keras.Model):
        def __init__(self, num_users, num_movies, embedding_size, **kwargs):
            super(RecommenderNet, self).__init__(**kwargs)
            self.num_users = num_users
            self.num_movies = num_movies
            self.embedding_size = embedding_size
            self.user_embedding = layers.Embedding(
                num_users,
                embedding_size,
                embeddings_initializer="he_normal",
                embeddings_regularizer=keras.regularizers.l2(1e-6),
            )
            self.user_bias = layers.Embedding(num_users, 1)
            self.movie_embedding = layers.Embedding(
            #change this line from num_movies to max_movie_index+1
                max_movie_index+1,
                embedding_size,
                embeddings_initializer="he_normal",
                embeddings_regularizer=keras.regularizers.l2(1e-6),
            )
            #and this line (input_dim)
            self.movie_bias = layers.Embedding(max_movie_index+1, 1)

        def call(self, inputs):
            user_vector = self.user_embedding(inputs[:, 0])
            user_bias = self.user_bias(inputs[:, 0])
            movie_vector = self.movie_embedding(inputs[:, 1])
            movie_bias = self.movie_bias(inputs[:, 1])
            dot_user_movie = tf.tensordot(user_vector, movie_vector, 2)
            # Add all the components (including bias)
            x = dot_user_movie + user_bias + movie_bias
            # The sigmoid activation forces the rating to between 0 and 1
            return tf.nn.sigmoid(x)


model = RecommenderNet(num_users, num_movies, EMBEDDING_SIZE)
model.compile(
        loss=tf.keras.losses.BinaryCrossentropy(), optimizer=keras.optimizers.Adam(lr=0.001)
    )
ratings = model.predict(that).flatten()

top_ratings_indices = ratings.argsort()[-10:][::-1]
movies_not_watched.loc[movies_not_watched['index'].isin(top_ratings_indices)]

<ipython-input-6-c862986cb048>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_not_watched['fake_id'] = np.ones(len(movies_not_watched), dtype =int)
<ipython-input-6-c862986cb048>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_all_ratings["movie"] = not_all_ratings["movieId"].map(dl_movie2movie_encoded)
<ipython-input-6-c862986cb048>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

Number of users: 610, Number of Movies: 9721, Min rating: 0.5, Max rating: 5.0


,index,movieId,title,genres,fake_id
6256,6256,46976,Stranger than Fiction (2006),Comedy|Drama|Fantasy|Romance,1
6260,6260,47124,"Ant Bully, The (2006)",Adventure|Animation|Children|Comedy|Fantasy|IMAX,1
7018,7018,68358,Star Trek (2009),Action|Adventure|Sci-Fi|IMAX,1
7027,7027,68600,Balls Out: Gary the Tennis Coach (2009),Comedy,1
7032,7032,68835,Were the World Mine (2008),Adventure|Fantasy|Musical|Romance,1
7657,7657,88515,Blitz (2011),Action|Crime|Thriller,1
7663,7663,88746,Terri (2011),Comedy,1
7669,7669,88954,"Very Harold & Kumar 3D Christmas, A (2011)",Comedy,1
7670,7670,89028,Don't Be Afraid of the Dark (2010),Horror|Thriller,1
7674,7674,89072,Stake Land (2010),Horror,1


In [7]:
ratings = model.predict(that).flatten()

top_ratings_indices = ratings.argsort()[-10:][::-1]
movies_not_watched.loc[movies_not_watched['index'].isin(top_ratings_indices)]

,index,movieId,title,genres,fake_id
6256,6256,46976,Stranger than Fiction (2006),Comedy|Drama|Fantasy|Romance,1
6260,6260,47124,"Ant Bully, The (2006)",Adventure|Animation|Children|Comedy|Fantasy|IMAX,1
7018,7018,68358,Star Trek (2009),Action|Adventure|Sci-Fi|IMAX,1
7027,7027,68600,Balls Out: Gary the Tennis Coach (2009),Comedy,1
7032,7032,68835,Were the World Mine (2008),Adventure|Fantasy|Musical|Romance,1
7657,7657,88515,Blitz (2011),Action|Crime|Thriller,1
7663,7663,88746,Terri (2011),Comedy,1
7669,7669,88954,"Very Harold & Kumar 3D Christmas, A (2011)",Comedy,1
7670,7670,89028,Don't Be Afraid of the Dark (2010),Horror|Thriller,1
7674,7674,89072,Stake Land (2010),Horror,1


In [8]:
not_all_ratings["movie"] = not_all_ratings["movieId"].map(dl_movie2movie_encoded)
not_all_ratings["movie"]

<ipython-input-8-f44992cb78f1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_all_ratings["movie"] = not_all_ratings["movieId"].map(dl_movie2movie_encoded)


0            0
1            1
2            2
3            3
4            4
          ... 
100831    3117
100832    2032
100833    3118
100834    1389
100835    2870
Name: movie, Length: 100474, dtype: int64

In [12]:
not_all_ratings[80000:80020]

,userId,movieId,title,rating,movie,user
80294,506,41571,Memoirs of a Geisha (2005),3.0,3370,505
80295,506,45447,"Da Vinci Code, The (2006)",0.5,1893,505
80296,506,45720,"Devil Wears Prada, The (2006)",3.0,3381,505
80297,506,45880,Marie Antoinette (2006),3.0,2801,505
80298,506,47629,The Queen (2006),4.0,2308,505
80299,506,48997,Perfume: The Story of a Murderer (2006),3.5,820,505
80300,506,49286,"Holiday, The (2006)",1.0,823,505
80301,506,51834,Becoming Jane (2007),3.0,899,505
80302,506,52973,Knocked Up (2007),3.5,1901,505
80303,506,52975,Hairspray (2007),3.0,3403,505


In [13]:
not_all_ratings.loc[not_all_ratings['movie'] == 3150]

,userId,movieId,title,rating,movie,user
9371,63,54259,Stardust (2007),3.5,3150,62
11356,68,54259,Stardust (2007),4.0,3150,67
16181,104,54259,Stardust (2007),4.0,3150,103
16623,105,54259,Stardust (2007),3.0,3150,104
21017,139,54259,Stardust (2007),2.5,3150,138
32012,219,54259,Stardust (2007),4.5,3150,218
33603,227,54259,Stardust (2007),2.5,3150,226
34607,232,54259,Stardust (2007),4.0,3150,231
35568,239,54259,Stardust (2007),4.0,3150,238
40401,274,54259,Stardust (2007),3.0,3150,273


In [14]:
all_ratings

,userId,movieId,title,rating
0,1,1,Toy Story (1995),4.0
1,1,3,Grumpier Old Men (1995),4.0
2,1,6,Heat (1995),4.0
3,1,47,Seven (a.k.a. Se7en) (1995),5.0
4,1,50,"Usual Suspects, The (1995)",5.0
...,...,...,...,...
100831,610,166534,Split (2017),4.0
100832,610,168248,John Wick: Chapter Two (2017),5.0
100833,610,168250,Get Out (2017),5.0
100834,610,168252,Logan (2017),5.0


In [ ]:
Try using .loc[row_indexer,col_indexer] = value instead

not_all_ratings.loc[not_all_ratings['movie'] == 3150]

In [ ]:
movies_not_watched['fake_id'] = np.ones(len(movies_not_watched), dtype =int)

In [ ]:
  not_all_ratings["movie"] = not_all_ratings["movieId"].map(dl_movie2movie_encoded)

In [18]:
not_all_ratings["movie"] = not_all_ratings.loc[not_all_ratings["movieId"], dl_movie2movie_encoded]
not_all_ratings["movie"]

not_all_ratings.loc[:, ('movie','movieId')] 

KeyError: "Passing list-likes to .loc or [] with any missing labels is no longer supported. The following labels were missing: Int64Index([   552, 106782, 112552, 114060, 115713,\n            ...\n            166534, 168248, 168250, 168252, 170875],\n           dtype='int64', length=6063). See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike"

In [20]:
#df.loc[:,'new column'] = value

not_all_ratings.loc[:, 'movie'] = not_all_ratings["movieId"]
not_all_ratings.loc[:, 'movie'][80000:80020]

80294    41571
80295    45447
80296    45720
80297    45880
80298    47629
80299    48997
80300    49286
80301    51834
80302    52973
80303    52975
80304    53000
80305    53125
80306    53894
80307    54259
80308    54272
80309    55052
80310    56367
80311    56782
80312    56949
80313    58154
Name: movie, dtype: int64

In [24]:
not_all_ratings.loc[not_all_ratings.loc[:, 'movieId'] == 47629]

,userId,movieId,title,rating,movie,user
4631,28,47629,The Queen (2006),2.0,47629,27
12167,74,47629,The Queen (2006),4.0,47629,73
16169,104,47629,The Queen (2006),4.0,47629,103
17257,110,47629,The Queen (2006),4.5,47629,109
25552,177,47629,The Queen (2006),4.0,47629,176
37670,254,47629,The Queen (2006),2.0,47629,253
50076,322,47629,The Queen (2006),4.0,47629,321
64549,414,47629,The Queen (2006),4.0,47629,413
75145,474,47629,The Queen (2006),4.0,47629,473
80298,506,47629,The Queen (2006),4.0,47629,505


In [50]:
#not_all_ratings.loc[not_all_ratings.loc[:, 'movieId'] == 47629]
#not_all_ratings.loc[:, 'movie'] = movies_not_watched["index"]
#เอา value มาใส่เลยไม่ได้ เพราะมันไม่เท่ากัน ต้อง map มาก่อนรอบนึง map ยังไงนะ จะเอา dl_movie2movie_encoded มาใช้ใน
#การสร้างคอลัมน์ใหม่ด้วย df.loc ยังไง เพราะ dataframe ไม่มี .map()

#not_all_ratings['movie'] = movies_not_watched["index"]
#df.loc[:,'new column'] = value

not_all_ratings.loc[:, 'movie'] = not_all_ratings["movieId"].map(dl_movie2movie_encoded)

#not_all_ratings["movie"] = not_all_ratings["movieId"].map(dl_movie2movie_encoded)
#ขอไม่เอา float ได้ไหม

TypeError: cannot convert the series to <class 'int'>

In [45]:
not_all_ratings[9690:9710]

,userId,movieId,title,rating,movie,user
9732,64.0,3396.0,"Muppet Movie, The (1979)",4.0,1708.0,63.0
9733,64.0,3408.0,Erin Brockovich (2000),2.0,420.0,63.0
9734,64.0,3418.0,Thelma & Louise (1991),3.5,1713.0,63.0
9735,64.0,3421.0,Animal House (1978),4.5,1714.0,63.0
9736,64.0,3450.0,Grumpy Old Men (1993),3.5,215.0,63.0
9737,64.0,3481.0,High Fidelity (2000),4.0,421.0,63.0
9738,64.0,3489.0,Hook (1991),3.0,217.0,63.0
9739,64.0,3499.0,Misery (1990),4.5,1036.0,63.0
9740,64.0,3510.0,Frequency (2000),3.0,1037.0,63.0
9741,64.0,3526.0,Parenthood (1989),4.0,1728.0,63.0


In [51]:
not_all_ratings

,userId,movieId,title,rating,movie,user
0,1.0,1.0,Toy Story (1995),4.0,0.0,0.0
1,1.0,3.0,Grumpier Old Men (1995),4.0,1.0,0.0
2,1.0,6.0,Heat (1995),4.0,2.0,0.0
3,1.0,47.0,Seven (a.k.a. Se7en) (1995),5.0,3.0,0.0
4,1.0,50.0,"Usual Suspects, The (1995)",5.0,4.0,0.0
...,...,...,...,...,...,...
100832,610.0,168248.0,John Wick: Chapter Two (2017),5.0,2032.0,609.0
100833,610.0,168250.0,Get Out (2017),5.0,3118.0,609.0
100834,610.0,168252.0,Logan (2017),5.0,1389.0,609.0
100835,610.0,170875.0,The Fate of the Furious (2017),3.0,2870.0,609.0


In [42]:
not_all_ratings[9690:9710]

,userId,movieId,title,rating,movie,user
9732,64.0,3396.0,"Muppet Movie, The (1979)",4.0,9732.0,63.0
9733,64.0,3408.0,Erin Brockovich (2000),2.0,9733.0,63.0
9734,64.0,3418.0,Thelma & Louise (1991),3.5,9734.0,63.0
9735,64.0,3421.0,Animal House (1978),4.5,9735.0,63.0
9736,64.0,3450.0,Grumpy Old Men (1993),3.5,9736.0,63.0
9737,64.0,3481.0,High Fidelity (2000),4.0,9737.0,63.0
9738,64.0,3489.0,Hook (1991),3.0,9738.0,63.0
9739,64.0,3499.0,Misery (1990),4.5,9739.0,63.0
9740,64.0,3510.0,Frequency (2000),3.0,9740.0,63.0
9741,64.0,3526.0,Parenthood (1989),4.0,9741.0,63.0


In [31]:
not_all_ratings[80000:80020]

,userId,movieId,title,rating,movie,user
80294,506,41571,Memoirs of a Geisha (2005),3.0,NaN,505
80295,506,45447,"Da Vinci Code, The (2006)",0.5,NaN,505
80296,506,45720,"Devil Wears Prada, The (2006)",3.0,NaN,505
80297,506,45880,Marie Antoinette (2006),3.0,NaN,505
80298,506,47629,The Queen (2006),4.0,NaN,505
80299,506,48997,Perfume: The Story of a Murderer (2006),3.5,NaN,505
80300,506,49286,"Holiday, The (2006)",1.0,NaN,505
80301,506,51834,Becoming Jane (2007),3.0,NaN,505
80302,506,52973,Knocked Up (2007),3.5,NaN,505
80303,506,52975,Hairspray (2007),3.0,NaN,505


In [28]:
movies_not_watched

,index,movieId,title,genres,fake_id
0,0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1
1,1,2,Jumanji (1995),Adventure|Children|Fantasy,1
2,2,3,Grumpier Old Men (1995),Comedy|Romance,1
3,3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1
4,4,5,Father of the Bride Part II (1995),Comedy,1
...,...,...,...,...,...
9737,9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,1
9738,9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,1
9739,9739,193585,Flint (2017),Drama,1
9740,9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,1


In [ ]:
  not_all_ratings["rating"] = not_all_ratings["rating"].values.astype(np.float32)

In [ ]:
  not_all_ratings["user"] = not_all_ratings["userId"].map(dl_user2user_encoded)